In [2]:
import socket
from confluent_kafka import Producer
import pymssql
import arrow
import sys
import airflow
from airflow.operators.python_operator import PythonOperator
from datetime import datetime

ModuleNotFoundError: No module named 'confluent_kafka'

In [ ]:
query = '''SELECT
COLUMN_NAME name,
DATA_TYPE type
FROM INFORMATION_SCHEMA.COLUMNS
where table_schema = 'dbo'
and table_name = 'Customer'
for json path;'''

In [1]:
import avro.schema
from avro.datafile import DataFileReader, DataFileWriter
from avro.io import DatumReader, DatumWriter

# schema = avro.schema.parse(open("user.avsc", "rb").read())

writer = DataFileWriter(open("users.avro", "wb"), DatumWriter(), schema)
writer.append({"name": "Alyssa", "favorite_number": 256})
writer.append({"name": "Ben", "favorite_number": 7, "favorite_color": "red"})
writer.close()

reader = DataFileReader(open("users.avro", "rb"), DatumReader())
for user in reader:
    print(user)
reader.close()

FileNotFoundError: [Errno 2] No such file or directory: 'user.avsc'

In [ ]:
topic = 'order-dt'

#server = "10.0.37.103:1433"  # dev-sql-31
#user = "test_user"
#password = "1qaz@WSX"
# database = "800Contacts"
date_filter = arrow.utcnow().shift(days=-1).to('US/Mountain').format('YYYY-MM-DD HH:mm:ss')

#conn = pymssql.connect(server, user, password, database)
conn = pymssql.connect(
    server='10.0.37.103:1433',
    user='test_user',
    password='1qaz@WSX',
    database='800Contacts'
)


default_args = {
    'owner': 'jaredrules!',
    'depends_on_past': False,
    'provide_context': True,
    'start_date': datetime(2019, 2, 20)
}

dag = airflow.DAG('jared-test', default_args=default_args)


def funct(**kwargs):
    with conn.cursor() as curs:
        try:
            curs.execute(f"select * from dbo.orderdt where orderdate >= '{date_filter}'")
            results = curs.fetchall()
        except BaseException as e:
            print(e)
        finally:
            conn.commit()
            conn.close()

    def delivery_callback(err, msg):
        if err:
            sys.stderr.write('%% Message failed delivery: %s\n' % err)
        else:
            sys.stderr.write('%% Message delivered to %s [%d] @ %d\n' %
                             (msg.topic(), msg.partition(), msg.offset()))

    p = Producer({'bootstrap.servers': "10.250.248.40:9092", 'client.id': socket.gethostname()})
    for i in results:
        try:
            p.produce(topic=topic, value=str(i), callback=delivery_callback)
            print(f'message sent {i}')
        except BufferError:
            sys.stderr.write(f'Local producer queue is full ({len(p)} messages awaiting delivery): try again\n')
        p.poll(0)
        sys.stderr.write(f'Waiting for {len(p)} deliveries\n')
    p.flush()


first_dag = PythonOperator(
    task_id='order-dt',
    python_callable=funct,
    dag=dag,
)

first_dag